In [1]:
# === Pass-Defense Index (NY/D only) with verification — WINDOWS-SAFE, ARROW-OPTIONAL ===
import os, sys, subprocess
import pandas as pd
import numpy as np

# --- A) Quiet rpy2 CFFI noise on Windows and ensure rpy2 is present ---
os.environ.setdefault("RPY2_CFFI_MODE", "ABI")
try:
    import rpy2  # noqa: F401
except ImportError:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "rpy2"])
from rpy2.robjects import r

# ------------------ CONFIG ------------------
OUT_DIR = "team_pass_defense"
os.makedirs(OUT_DIR, exist_ok=True)
seasons = list(range(2000, 2025))  # inclusive 2000–2024
MIN_DROPBACKS = 200                 # sample-size screen
# PDX-4 weights (efficiency pillar will be NY/D)
W_EFF, W_EXPL, W_COMP, W_INT = 0.40, 0.25, 0.20, 0.15
# --------------------------------------------

# --- B) R deps & options (prefer parquet if arrow is available) ---
OUT_DIR_R = OUT_DIR.replace("\\", "/")
r(f"""
deps <- c("nflreadr","dplyr","tidyr","readr","arrow")
to_install <- deps[!vapply(deps, requireNamespace, FUN.VALUE=logical(1), quietly=TRUE)]
if (length(to_install)) install.packages(to_install, repos="https://cloud.r-project.org")

suppressPackageStartupMessages({{
  library(nflreadr); library(dplyr); library(tidyr); library(readr)
}})

USE_PARQUET <- requireNamespace("arrow", quietly = TRUE)
options(nflreadr.prefer = if (USE_PARQUET) "parquet" else "csv")
options(nflreadr.verbose = FALSE)

# Clear nflreadr cache once (protects against stale results)
try(nflreadr::.clear_cache(), silent = TRUE)
""")

# --- C) Per-season extraction → CSVs (EPA ignored entirely) ---
for season in seasons:
    print(f"  → {season} (auto file_type: parquet if arrow present, else csv)")
    r.assign("SEASON", season)
    r.assign("OUT_DIR", OUT_DIR_R)
    r("""
library(nflreadr); library(dplyr); library(tidyr); library(readr)

file_type <- if (getOption("nflreadr.prefer","csv") == "parquet") "parquet" else "csv"
message(sprintf("Loading PBP for %s via %s …", SEASON, file_type))

pbp <- nflreadr::load_pbp(seasons = SEASON, file_type = file_type)

# Guard: some seasons could be empty if something’s off
if (is.null(pbp) || !nrow(pbp)) {
  stop(sprintf("No rows returned for season %s. Try clearing cache and/or switching file_type.", SEASON))
}

pbp <- pbp %>% dplyr::filter(!is.na(defteam))

# Helper to ensure flags exist
add_if_missing_flag <- function(df, col) {
  if (!col %in% names(df)) df[[col]] <- 0L
  df[[col]] <- tidyr::replace_na(df[[col]], 0L)
  df
}

# Normalize common flags across eras
for (nm in c("pass","complete_pass","sack","scramble","interception")) {
  pbp <- add_if_missing_flag(pbp, nm)
}
if (!"qb_dropback" %in% names(pbp)) pbp$qb_dropback <- NA

# Robust yards fields
if (!"sack_yards" %in% names(pbp)) pbp$sack_yards <- 0
pbp$sack_yards <- dplyr::if_else(is.na(pbp$sack_yards), 0, pbp$sack_yards)
if (!"yards_gained" %in% names(pbp)) pbp$yards_gained <- 0

pbp <- pbp %>%
  dplyr::mutate(
    is_attempt  = as.integer(pass),
    is_complete = as.integer(complete_pass),
    is_sack     = as.integer(sack),
    is_scramble = as.integer(scramble),
    is_int      = as.integer(interception),

    qb_db_int   = dplyr::case_when(
      !is.na(qb_dropback) ~ as.integer(qb_dropback),
      TRUE ~ as.integer((is_attempt==1L) | (is_sack==1L) | (is_scramble==1L))
    ),
    is_dropback = qb_db_int,

    is_explosive = dplyr::if_else(is_complete==1L & !is.na(yards_gained) & yards_gained >= 20, 1L, 0L),

    net_yards_db_component =
      dplyr::if_else(is_attempt==1L, yards_gained, 0) +
      dplyr::if_else(is_sack==1L, -abs(sack_yards), 0) +
      dplyr::if_else(is_scramble==1L, yards_gained, 0)
  )

team_def <- pbp %>%
  dplyr::filter(is_dropback == 1L) %>%
  dplyr::group_by(season, defteam) %>%
  dplyr::summarise(
    dropbacks   = dplyr::n(),
    attempts    = sum(is_attempt,  na.rm = TRUE),
    completions = sum(is_complete, na.rm = TRUE),
    sacks       = sum(is_sack,     na.rm = TRUE),
    scrambles   = sum(is_scramble, na.rm = TRUE),
    explosive_comps = sum(is_explosive, na.rm = TRUE),
    interceptions   = sum(is_int,       na.rm = TRUE),
    net_yards_sum = sum(net_yards_db_component, na.rm = TRUE),
    .groups = "drop_last"
  ) %>%
  dplyr::mutate(
    net_yards_per_db_allowed    = dplyr::if_else(dropbacks > 0, net_yards_sum / dropbacks, NA_real_),
    explosive_pass_rate_allowed = dplyr::if_else(completions > 0, explosive_comps / completions, NA_real_),
    completion_pct_allowed      = dplyr::if_else(attempts > 0, completions / attempts, NA_real_),
    int_rate_allowed            = dplyr::if_else(attempts > 0, interceptions / attempts, NA_real_)
  ) %>%
  dplyr::arrange(defteam)

message(sprintf("Season %s: teams=%d, dropbacks[sum]=%d",
        SEASON, dplyr::n_distinct(team_def$defteam), sum(team_def$dropbacks, na.rm = TRUE)))

readr::write_csv(team_def, file.path(OUT_DIR, paste0("team_pass_def_", SEASON, ".csv")))
""")

# --- D) Combine per-season CSVs (raw aggregates) ---
all_files = [os.path.join(OUT_DIR, f) for f in os.listdir(OUT_DIR) if f.endswith(".csv")]
if not all_files:
    raise RuntimeError("No per-season CSVs were produced. Check the R loop above.")

missing = [yr for yr in seasons if not any(f.endswith(f"team_pass_def_{yr}.csv") for f in all_files)]
if missing:
    raise RuntimeError(
        f"Missing per-season CSVs for: {missing}. "
        "If you're on Windows and don't have R 'arrow', parquet may fail—this code already falls back to CSV. "
        "If the error persists, re-run to rebuild cache."
    )

df_all = pd.concat((pd.read_csv(f) for f in sorted(all_files)), ignore_index=True)

print("Combined shape:", df_all.shape)
print("Teams per season (first 5 seasons):")
print(df_all.groupby("season")["defteam"].nunique().head())

# Order & write the raw per-season aggregates
expected_cols = [
    "season","defteam","dropbacks","attempts","completions","sacks","scrambles",
    "explosive_comps","interceptions","net_yards_sum",
    "net_yards_per_db_allowed",
    "explosive_pass_rate_allowed","completion_pct_allowed","int_rate_allowed"
]
cols = [c for c in expected_cols if c in df_all.columns] + [c for c in df_all.columns if c not in expected_cols]
df_all = df_all[cols]
df_all.to_csv("team_pass_defense_all.csv", index=False)
print("✅ Saved team_pass_defense_all.csv")

# --- E) Build PDX-4 (NY/D as efficiency pillar) + corrected percentile ---
df = df_all.copy()

# Screen tiny samples
mask_small = df["dropbacks"] < MIN_DROPBACKS
df.loc[mask_small, ["net_yards_per_db_allowed",
                    "explosive_pass_rate_allowed","completion_pct_allowed","int_rate_allowed"]] = np.nan

def zscore_group(s: pd.Series) -> pd.Series:
    mu = s.mean(skipna=True)
    sd = s.std(skipna=True, ddof=0)
    if pd.isna(sd) or sd == 0:
        return pd.Series(np.zeros(len(s), dtype="float64"), index=s.index)
    return (s - mu) / sd

# Efficiency pillar (NY/D): lower allowed is better
df["eff_value"] = df["net_yards_per_db_allowed"].astype(float)

# Z-scores within season
df["z_eff"]  = df.groupby("season")["eff_value"].transform(zscore_group)                   # lower better → flip
df["z_expl"] = df.groupby("season")["explosive_pass_rate_allowed"].transform(zscore_group) # lower better → flip
df["z_comp"] = df.groupby("season")["completion_pct_allowed"].transform(zscore_group)      # lower better → flip
df["z_int"]  = df.groupby("season")["int_rate_allowed"].transform(zscore_group)            # higher better

# Flip where lower is better
df["z_eff_f"]  = -df["z_eff"]
df["z_expl_f"] = -df["z_expl"]
df["z_comp_f"] = -df["z_comp"]
df["z_int_f"]  =  df["z_int"]

# Composite
df["PDX4"] = (W_EFF*df["z_eff_f"] + W_EXPL*df["z_expl_f"] + W_COMP*df["z_comp_f"] + W_INT*df["z_int_f"])

# Rank (1=best) and Strength Percentile (1=best, 0=worst)
df["season_rank"] = df.groupby("season")["PDX4"].rank(ascending=False, method="min")
df["_n_season"] = df.groupby("season")["PDX4"].transform("count")
df["season_pct"] = np.where(
    df["_n_season"] > 1,
    1 - (df["season_rank"] - 1) / (df["_n_season"] - 1),
    1.0
)
df = df.drop(columns=["_n_season"])

# Output by season (clean team col)
out_by_season = df[[
    "season","defteam","dropbacks","attempts","completions",
    "net_yards_per_db_allowed",
    "explosive_pass_rate_allowed","completion_pct_allowed","int_rate_allowed",
    "PDX4","season_rank","season_pct"
]].rename(columns={"defteam":"team"}).sort_values(["season","season_rank"])
out_by_season.to_csv("team_pass_defense_pdx4_by_season.csv", index=False)

# Pre-draft table: season t–1 → draft year t
pre = df.copy()
pre["draft_year"] = pre["season"] + 1
pre = pre.rename(columns={
    "defteam": "team",
    "PDX4": "pdx4_prev",
    "season_rank": "rank_prev",
    "season_pct": "pct_prev"
})
pre_cols = ["draft_year","team","pdx4_prev","rank_prev","pct_prev",
            "eff_value","explosive_pass_rate_allowed",
            "completion_pct_allowed","int_rate_allowed","dropbacks"]
pre = pre[pre_cols].sort_values(["draft_year","rank_prev"])
pre.to_csv("pre_draft_db_strength.csv", index=False)

# Slim positional rank CSVs
db_pos_rank_by_season = out_by_season[["season","team","PDX4","season_rank","season_pct"]]
db_pos_rank_by_season.to_csv("db_pos_rank_by_season.csv", index=False)

pre_draft_db_pos_rank = pre[["draft_year","team","pdx4_prev","rank_prev","pct_prev"]]
pre_draft_db_pos_rank.to_csv("pre_draft_db_pos_rank.csv", index=False)

# --- F) VERIFICATION SUITE ---
problems = []

# 1) Monotonicity checks
g = out_by_season.copy()
g["rank_diff"] = g.groupby("season")["season_rank"].diff()
g["pdx_diff"]  = g.groupby("season")["PDX4"].diff()
g["pct_diff"]  = g.groupby("season")["season_pct"].diff()

viol_pdx = ((g["rank_diff"] > 0) & (g["pdx_diff"] > 1e-12)).sum()
viol_pct = ((g["rank_diff"] > 0) & (g["pct_diff"] > 1e-12)).sum()
if viol_pdx > 0: problems.append(f"Monotonicity violation: PDX4 increases with worse rank in {viol_pdx} rows.")
if viol_pct > 0: problems.append(f"Monotonicity violation: percentile increases with worse rank in {viol_pct} rows.")

# 2) Percentile scaling
rng = out_by_season.groupby("season")["season_pct"].agg(["min","max"])
bad_rng = rng[(rng["min"] < -1e-9) | (rng["max"] > 1 + 1e-9)]
if not bad_rng.empty:
    problems.append(f"Percentile out of [0,1] in seasons: {bad_rng.index.tolist()}")

# 3) Sample size screen honored
if (df["dropbacks"] < MIN_DROPBACKS).any():
    cols_to_check = ["eff_value","explosive_pass_rate_allowed","completion_pct_allowed","int_rate_allowed"]
    bad = df.loc[df["dropbacks"] < MIN_DROPBACKS, cols_to_check].notna().any(axis=1).sum()
    if bad > 0:
        problems.append("Tiny-sample rows still have non-NaN pillar values.")

# 4) Positive correlation of flipped z-pillars with PDX4
corrs = df[["z_eff_f","z_expl_f","z_comp_f","z_int_f","PDX4"]].corr().loc[
    ["z_eff_f","z_expl_f","z_comp_f","z_int_f"], "PDX4"
]
if (corrs < -0.05).any():
    problems.append(f"Unexpected negative correlation(s) with PDX4: {corrs[corrs < -0.05].to_dict()}")

# Validation report
val_rows = []
for s, sub in out_by_season.groupby("season"):
    val_rows.append({
        "season": int(s),
        "teams_reported": int(sub["team"].nunique()),
        "pdx4_std": float(sub["PDX4"].std(ddof=0)),
        "nyd_mean": float(sub["net_yards_per_db_allowed"].mean()),
        "nyd_std": float(sub["net_yards_per_db_allowed"].std(ddof=0)),
        "expl_rate_mean": float(sub["explosive_pass_rate_allowed"].mean()),
        "comp_pct_mean": float(sub["completion_pct_allowed"].mean()),
        "int_rate_mean": float(sub["int_rate_allowed"].mean()),
        "pct_min": float(sub["season_pct"].min()),
        "pct_max": float(sub["season_pct"].max())
    })
pd.DataFrame(val_rows).to_csv("pass_defense_validation_report.csv", index=False)

if problems:
    print("\n❗ VALIDATION WARNINGS:")
    for p in problems: print(" -", p)
else:
    print("\n✅ Validation passed (monotonicity, scaling, screening, correlations).")

print("\nSample (top team per season):")
print(out_by_season.groupby("season").head(1)[["season","team","PDX4","season_rank","season_pct"]].head(10))

print("\n✅ Outputs:")
print(" - team_pass_defense_all.csv")
print(" - team_pass_defense_pdx4_by_season.csv  (percentile: 1=best)")
print(" - pre_draft_db_strength.csv             (pct_prev: 1=best)")
print(" - db_pos_rank_by_season.csv")
print(" - pre_draft_db_pos_rank.csv")
print(" - pass_defense_validation_report.csv    (for CMU verification)")


v nflreadr function cache cleared!


R callback write-console: <class 'UnicodeDecodeError'> 'utf-8' codec can't decode byte 0x85 in position 33: invalid start byte <traceback object at 0x000001F76E363B00>


  → 2000 (auto file_type: parquet if arrow present, else csv)


R callback write-console: Season 2000: teams=31, dropbacks[sum]=18767
  
R callback write-console: <class 'UnicodeDecodeError'> 'utf-8' codec can't decode byte 0x85 in position 33: invalid start byte <traceback object at 0x000001F76E363800>


  → 2001 (auto file_type: parquet if arrow present, else csv)


...

  → 2023 (auto file_type: parquet if arrow present, else csv)


R callback write-console: Season 2023: teams=32, dropbacks[sum]=21819
  
R callback write-console: <class 'UnicodeDecodeError'> 'utf-8' codec can't decode byte 0x85 in position 33: invalid start byte <traceback object at 0x000001F76E39CB00>


  → 2024 (auto file_type: parquet if arrow present, else csv)


R callback write-console: Season 2024: teams=32, dropbacks[sum]=21139
  


Combined shape: (798, 14)
Teams per season (first 5 seasons):
season
2000    31
2001    31
2002    32
2003    32
2004    32
Name: defteam, dtype: int64
✅ Saved team_pass_defense_all.csv

✅ Validation passed (monotonicity, scaling, screening, correlations).

Sample (top team per season):
     season team      PDX4  season_rank  season_pct
2      2000  BAL  1.454036          1.0         1.0
38     2001  CLE  1.394187          1.0         1.0
91     2002   TB  1.959837          1.0         1.0
96     2003  BAL  1.571158          1.0         1.0
129    2004  BUF  1.692494          1.0         1.0
163    2005  CHI  1.344553          1.0         1.0
195    2006  CHI  1.287966          1.0         1.0
251    2007   TB  1.255222          1.0         1.0
280    2008  PIT  1.717171          1.0         1.0
310    2009  NYJ  1.469898          1.0         1.0

✅ Outputs:
 - team_pass_defense_all.csv
 - team_pass_defense_pdx4_by_season.csv  (percentile: 1=best)
 - pre_draft_db_strength.csv         